<h1 style="text-align:center;">Лабораторная работа №6</h2>

<h2 style="text-align:center;">Решение жесткой системы однородных  дифференциальных уравнений явными методами</h2>

<h3 style="text-align:center;">Тулупов Николай Дмитриевич</h3>
<h3 style="text-align:center;">Б01-204</h3>

Цель: решить выбранную задачу перечисленными методами:   

0) Попробовать явный метод из лабы 2
1) Неявные методы Рунге-Кутта 3+ порядка
2) Невные методы Адамса / ФДН (методы Гира) 3+ порядка
3) Метод Розенброка-Ваннера 2, 3+ порядка   
4) методы Гира в представлении Нордсика 
     
<img src="images\task.png" alt="drawing" width="750"/>

**Чем отличаются жесткие системы?**   
 У них есть быстро затухающие компоненты и медленно меняющиеся одновременно. Поэтому при решении обычными (явными) методами нужно брать очень маленький шаг, иначе решение разлетается.

**Почему неявные методы лучше для жёстких?**   
Потому что они устойчивы при больших шагах даже для "резких" уравнений!




In [205]:
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt

In [206]:
t_start = 0
t_stop = 20
Y0 = [0.5,  0,    0,    np.sqrt(3)]
#     x(0)  y(0)  z(0)  u(0)

In [207]:
def f(t, Y):
    x, y, z, u = Y
    r = (x**2 + y**2)**1.5
    return [z, u, -x / r, -y / r]


### **Явные методы**

In [208]:
def runge_kutt_explicit(f, t, Y, h, order):
    l = len(Y)
    if order == 1:
        k1 = f(t, Y)
        return [ (Y[i] + h * k1[i])  for i in range(l)]
    
    elif order == 2:
        k1 = f(t, Y)
        Y_pred = [Y[i] + h * k1[i] for i in range(l)]  # прогноз
        k2 = f(t + h, Y_pred)
        return [Y[i] + h * (k1[i] + k2[i]) / 2 for i in range(l)]
    
    elif order == 3:
        k1 = f(t, Y)
        Y_pred_12 = [Y[i] + h * k1[i] / 2 for i in range(l)]
        k2 = f(t + h/2, Y_pred_12)
        Y_pred_23 =  [Y[i] - h * k1[i] + 2 * h * k2[i] for i in range(l)]
        k3 = f(t + h, Y_pred_23)
        return [Y[i] + h * (k1[i] + 4*k2[i] + k3[i]) / 6 for i in range(l)]
    
    elif order == 4:
        k1 = f(t, Y)
        Y_pred_12 = [Y[i] + h * k1[i] / 2 for i in range(l)]
        k2 = f(t + h / 2, Y_pred_12)
        Y_pred_23 = [Y[i] + h * k2[i] / 2 for i in range(l)]
        k3 = f(t + h / 2, Y_pred_23)
        Y_pred_34 = [Y[i] + h * k3[i] for i in range(l)]
        k4 = f(t + h, Y_pred_34)
        return [Y[i] + h * (k1[i] + 2*k2[i] + 2*k3[i] + k4[i]) / 6 for i in range(l)]
    
    else:
        print("ERROR! Порядок должен быть от 1 до 4")
        return 

In [209]:
def integrate_runge_kutt_explicit(f, Y0, t_start, t_stop, h, order):
    t = t_start
    Y = Y0[:]
    S = [[t] + Y] 

    while t < t_stop:
        Y = runge_kutt_explicit(f, t, Y, h, order)
        t += h
        S.append([t] + Y)

    return S #траектория

In [210]:
def drawing(method, order):
    h_vals = [10 ** i for i in range (-5, 1)]
    x_vals = [0]*6
    y_vals = [0]*6

    for i in range(len(h_vals)):
        S = method(f, Y0, t_start, t_stop, h_vals[i], order)
        x_vals[i] = [p[1] for p in S]
        y_vals[i] = [p[2] for p in S]
        #plt.title(f'Траектория движения спутника: (РК1), шаг = {h}')

    fig, axs = plt.subplots(2, 3, figsize=(15, 8))  # 2 строки, 3 столбца

    axs = axs.flatten()  # чтобы обращаться как axs[0], axs[1], ...

    for i in range(len(h_vals)):
        axs[i].plot(x_vals[i], y_vals[i])
        axs[i].set_title(f"Шаг h = {h_vals[i]}")
        axs[i].set_xlabel('x')
        axs[i].set_ylabel('y')
        axs[i].axis('equal')  # чтобы не искажалась орбита
        axs[i].grid(True)



    method_name = method.__name__

    # Красивое название (преобразуем имя функции)
    if "adams" in method_name:
        method_display = "Адамса"
    elif "fdn" in method_name:
        method_display = "ФДН"
    elif "runge" in method_name:
        method_display = "Рунге-Кутты"
    else:
        method_display = method_name  # fallback

    fig.suptitle(f"Траектория движения спутника\nМетод {method_display} {order} порядка", fontsize=16)
    plt.tight_layout()
    plt.show()